In [ ]:
# !Convert*multiscale_point_mosek.py*w*sh*

In [ ]:
import math

import numpy

import mosek

In [ ]:
def shrink_u(m, mu, mup, shrink, mele=0):
    mu_map = mup.max(axis=0)
    mu_mip = mup.min(axis=0)
    
    buc = [0] * m
    con = []
    for i in range(shrink**2):
        con.append([])
    
    for i in range(m):
        tmp_x = min(max(int(shrink * (mup[i, 0] - mu_mip[0]) / (mu_map[0] - mu_mip[0])), 0), shrink-1)
        tmp_y = min(max(int(shrink * (mup[i, 1] - mu_mip[1]) / (mu_map[1] - mu_mip[1])), 0), shrink-1)
        
        b = tmp_x*shrink + tmp_y
        
        buc[i] = b
        con[b].append(i)
    
    new_con = []
    
    for i in range(shrink**2):
        if len(con[i]) > mele:
            new_con.append(con[i])
    
    new_m = len(new_con)
    new_mu = numpy.zeros((new_m))
    
    rep = [None] * new_m
    for i in range(new_m):
        subset = new_con[i]
        buc_cen = mup[subset, :].mean(axis=0)
        rep[i] = subset[numpy.argmin(((mup[subset, :] - buc_cen)**2).sum(axis=1))]
        new_mu[i] = mu[subset].sum()
    
    new_mup = mup[rep]
    
    return new_con, rep, new_m, new_mu, new_mup

In [ ]:
def shrink_c(c, rep_m, rep_n):
    new_c = c[numpy.meshgrid(rep_m, rep_n)].transpose()
    return new_c

In [ ]:
def propagate(m, n, con_m, con_n, path_coarsen):
    path = []
    for p in path_coarsen:
        scale = len(con_m[p[1]]) * len(con_n[p[2]])
        for i in con_m[p[1]]:
            for j in con_n[p[2]]:
                path.append((
                    p[0] / scale,
                    i,
                    j
                ))
    return path

In [ ]:
def small(m, n, mu, nu, c, capacity, error, mtd, solt):
    scale = math.sqrt(m * n)
    
    mu, nu = mu*scale, nu*scale
    
    ris = numpy.minimum(mu.reshape((m, 1)), nu.reshape((1, n)))
    
    with mosek.Env() as env:
        with env.Task() as task:
            task.putintparam(mosek.iparam.optimizer, mtd)
            
            task.appendvars(m*n)
            task.appendcons(m+n)
            
            task.putvarboundlist(
                range(m*n),
                [mosek.boundkey.ra]*(m*n),
                [0.]*(m*n),
                (capacity[0] * ris).reshape((m*n))
            )
            
            for i in range(m):
                task.putarow(
                    i,
                    range(i*n, (i+1)*n),
                    [1.]*n
                )
            task.putconboundlist(
                range(0, m),
                [mosek.boundkey.fx]*m,
                mu,
                mu
            )
    
            for i in range(n):
                task.putarow(
                    i+m,
                    range(i, i+m*n, n),
                    [1.]*m
                )
            task.putconboundlist(
                range(m, m+n),
                [mosek.boundkey.fx]*n,
                nu,
                nu
            )
            
            task.putclist(range(m*n), c.reshape(m*n))
            task.putobjsense(mosek.objsense.minimize)
            task.optimize()
            
            sol = [0.] * (m*n)
            task.getxx(solt, sol)
    
    path = [(sol[i*n + j] / scale, i, j) for i in range(m) for j in range(n) if sol[i*n + j] > error[0] * ris[i, j]]

    return path

In [ ]:
def multi(m, n, mu, nu, mup, nup, c, step, shrink, capacity, error, mtd, solt):
    if step <= 1:
        return small(m, n, mu, nu, c, capacity, error, mtd, solt)
    
    con_m, rep_m, m_shrink, mu_shrink, mup_shrink = shrink_u(m, mu, mup, shrink[0])
    con_n, rep_n, n_shrink, nu_shrink, nup_shrink = shrink_u(n, nu, nup, shrink[0])
    c_shrink = shrink_c(c, rep_m, rep_n)
    
    path_coarsen = multi(m_shrink, n_shrink, mu_shrink, nu_shrink, mup_shrink, nup_shrink, c_shrink, step-1, shrink[1:], capacity[1:], error[1:], mtd, solt)
    path = propagate(m, n, con_m, con_n, path_coarsen)
    l = len(path)
    
    ris = numpy.minimum(mu.reshape((m, 1)), nu.reshape((1, n)))
    ris = ris[[p[1] for p in path], [p[2] for p in path]]
    
    with mosek.Env() as env:
        with env.Task() as task:
            task.putintparam(mosek.iparam.optimizer, mtd)
            
            task.appendvars(l)
            task.appendcons(m+n)
            
            task.putvarboundlist(
                range(l),
                [mosek.boundkey.ra]*(l),
                [0.]*(l),
                capacity[0] * ris
            )
            
            tmp1 = [[] for i in range(m)]
            tmp2 = [[] for i in range(n)]
            for j in range(l):
                tmp1[path[j][1]].append(j)
                tmp2[path[j][2]].append(j)
            
            for i in range(m):
                task.putarow(
                    i,
                    tmp1[i],
                    [1.] * len(tmp1[i])
                )
            task.putconboundlist(
                range(0, m),
                [mosek.boundkey.fx]*m,
                mu,
                mu
            )

            for i in range(n):
                task.putarow(
                    i+m,
                    tmp2[i],
                    [1.] * len(tmp2[i])
                )
            task.putconboundlist(
                range(m, m+n),
                [mosek.boundkey.fx]*n,
                nu,
                nu
            )
            
            task.putclist(range(l), c[[p[1] for p in path], [p[2] for p in path]])
            task.putobjsense(mosek.objsense.minimize)
            task.optimize()

            xx = [0.] * l
            task.getxx(solt, xx)

    new_path = []
    for i in range(l):
        if xx[i] > error[0] * ris[i]:
            new_path.append((
                xx[i],
                path[i][1],
                path[i][2]
            ))

    return new_path

In [ ]:
def solve_multiscale_point_mosek(
    p,
    step, shrinks, caps, errs, mtd, solt,
    log=None, stat=False, title="",
    *args, **kwargs
):
    m, n = p.c.shape
    
    path = multi(m, n, p.mu, p.nu, p.mup, p.nup, p.c, step, shrinks, caps, errs, mtd, solt)
    
    sol = numpy.zeros((m, n))
    for pa in path:
        sol[pa[1], pa[2]] = pa[0]
    
    p.s = sol
    
    if stat:
        s = {
            "title": title,
            "loss": (p.c * sol).sum(),
        }
        return p, s
    else:
        return p

In [ ]:
def solve_multiscale_point_mosek_interior_point(
    p,
    step, shrinks, caps, errs,
    *args, **kwargs
):
    return solve_multiscale_point_mosek(
        p, step, shrinks, caps, errs, 
        mtd=mosek.optimizertype.intpnt,
        solt=mosek.soltype.itr,
        title="Multiscale for image using mosek interior point",
        *args, **kwargs
    )

def solve_multiscale_point_mosek_primal_simplex(
    p,
    step, shrinks, caps, errs,
    *args, **kwargs
):
    return solve_multiscale_point_mosek(
        p, step, shrinks, caps, errs, 
        mtd=mosek.optimizertype.primal_simplex,
        solt=mosek.soltype.bas,
        title="Multiscale for image using mosek primal simplex",
        *args, **kwargs
    )

In [ ]:
# !ConvertEnd*

In [ ]:
# !Convert*multiscale_point_mosek_test.py*w*sehx*

In [ ]:
# !Switch*
from handler import FigureHandler
from dataset import ot_2d_Caffarelli, ot_2d_ellipse
from stats import Statistics
# !SwitchCase*
# import font
# from handler import FigureHandler
from dataset import ot_2d_Caffarelli, ot_2d_ellipse
# from stats import Statistics
# from multiscale_point_mosek import *
# !SwitchEnd*

In [ ]:
# !Switch*
fh = FigureHandler(redir=True)
# !SwitchCase*
# fh = FigureHandler(sav=True, disp=False, ext=".pgf", redir=True)
# !SwitchEnd*

In [ ]:
prob = ot_2d_ellipse(100, 100, 1)

In [ ]:
fh.fast(prob.plot_mu_scatter_plain)

In [ ]:
con_m, rep_m, new_m, new_mu, new_mup = shrink_u(prob.c.shape[0], prob.mu, prob.mup, 5, 0)

In [ ]:
con_n, rep_n, new_n, new_nu, new_nup = shrink_u(prob.c.shape[1], prob.nu, prob.nup, 5, 0)

In [ ]:
new_c = shrink_c(prob.c, rep_n, rep_m)

In [ ]:
prob.mup, prob.nup, prob.c = new_mup, new_nup, new_c

In [ ]:
fh.fast(prob.plot_mu_scatter_plain)

In [ ]:
stat = Statistics(
    probs=[
        ot_2d_Caffarelli(500, 500, 1)
    ],
    prob="Test problems",
    log=fh.write,
)

In [ ]:
stat.test(
    solve_multiscale_point_mosek_interior_point,
    step=4, shrinks=[50, 25, 10, 0],
    caps=[1., 0.3, 0.1, 0.1],
    errs=[0.001, 0.001, 0.001, 0.00],
    clean=False
)
stat.output_last()
fh.fast(stat.probs[0].plot_link)
stat.probs[0].clean()

In [ ]:
from solver_mosek import solve_mosek_interior_point

In [ ]:
stat.test(
    solve_mosek_interior_point,
    clean=False
)
stat.output_last()
fh.fast(stat.probs[0].plot_link)
stat.probs[0].clean()

In [ ]:
# !ConvertEnd*